In [1]:
import stanza, re, benepar, psutil, gc, json, csv, time, string, os

import numpy as np
import networkx as nx
import pandas as pd
import seaborn as sns

from textstat import textstat
from datetime import datetime
from collections import Counter

import spacy
from spacy.tokenizer import Tokenizer
from spacy.util import compile_infix_regex


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk import Tree
from textblob import Word
from spellchecker import SpellChecker

import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator, LogFormatter

In [ ]:
# These only need to run once
# stanza.download('en')
spacy.cli.download("en_core_web_sm")
nltk.download('punkt_tab')
benepar.download('benepar_en3')

In [2]:
source_texts= list()
source_texts.append("Kazuo Ishiguro - Never Let Me Go")
source_texts.append("Kazuo Ishiguro - The Remains of the Day")
source_texts.append("Kazuo Ishiguro - A Pale View of Hills-Knopf Doubleday Publishing Group (1990)")
source_texts.append("Kazuo-Ishiguro-When-We-Were-Orphans-Alfred-A.-Knopf_Vintage-_2001_")
source_texts.append("The Buried Giant (Kazuo Ishiguro) (Z-Library)-1")
source_texts.append("Kazuo Ishiguro - The Unconsoled-Vintage (1996)")
directory_path = "C:/Users/Roland/Documents/AI/stylometry/"


Select choice of text by specifying index (starting with 0):

In [3]:
text_choice = 4


In [4]:
file_path = directory_path+source_texts[text_choice]+".txt"
file_path_edited = file_path[:-4]+" edited.txt"
file_path_edited_1 = file_path[:-4]+" edited_1.txt"
file_path_edited_2 = file_path[:-4]+" edited_2.txt"
print (file_path)
print(file_path_edited)
print(file_path_edited_1)
print(file_path_edited_2)

C:/Users/Roland/Documents/AI/stylometry/The Buried Giant (Kazuo Ishiguro) (Z-Library)-1.txt
C:/Users/Roland/Documents/AI/stylometry/The Buried Giant (Kazuo Ishiguro) (Z-Library)-1 edited.txt
C:/Users/Roland/Documents/AI/stylometry/The Buried Giant (Kazuo Ishiguro) (Z-Library)-1 edited_1.txt
C:/Users/Roland/Documents/AI/stylometry/The Buried Giant (Kazuo Ishiguro) (Z-Library)-1 edited_2.txt


Try reading the file with different encodings:

In [5]:
encodings = ['utf-8', 'ascii', 'latin-1','windows-1252', 'iso-8859-1', 'iso-8859-15']
texts = []
valid_encodings = list()
for i, encoding in enumerate(encodings):
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            test_text = file.read()
        print("""Successfully read the file with encoding: {encoding} (encoding index {ind}).  This gives the beginning as:\n{content}\n""".format(encoding=encoding, ind=i, content=test_text[0:200].replace("\n", "  ")))
        texts.append(test_text)
        valid_encodings.append(i)
    except UnicodeDecodeError:
        print(f"Failed to read with encoding: {encoding}")
print("valid encodings: ",valid_encodings)

Successfully read the file with encoding: utf-8 (encoding index 0).  This gives the beginning as:
Part I                              Chapter One      You would have searched a long time for the sort of winding lane or  tranquil meadow for which England later became celebrated. There  were instead mile

Failed to read with encoding: ascii
Successfully read the file with encoding: latin-1 (encoding index 2).  This gives the beginning as:
Part I                              Chapter One      You would have searched a long time for the sort of winding lane or  tranquil meadow for which England later became celebrated. There  were instead mile

Failed to read with encoding: windows-1252
Successfully read the file with encoding: iso-8859-1 (encoding index 4).  This gives the beginning as:
Part I                              Chapter One      You would have searched a long time for the sort of winding lane or  tranquil meadow for which England later became celebrated. There  were instead mi

Are they the same?

In [6]:
for i in range(len(texts)):
    print (f"text {i} length {len(texts[i])}:-")
    for j, t in enumerate(texts):
        print (j, t == texts[i])

text 0 length 569311:-
0 True
1 False
2 False
3 False
text 1 length 584195:-
0 False
1 True
2 True
3 False
text 2 length 584195:-
0 False
1 True
2 True
3 False
text 3 length 584195:-
0 False
1 False
2 False
3 True


Now load text file with selected encoding

In [7]:
def check_for_file(file_path):
    if os.path.isfile(file_path):
        print("The file exists.")
    else:
        print("The file does not exist.")
    


In [8]:
check_for_file(file_path_edited)

The file does not exist.


In [9]:
for t in texts:
    with open(file_path_edited, 'w', encoding='utf-8') as file:
        file.write(t)
    with open(file_path_edited, 'r', encoding='utf-8') as file:
        text_read = file.read()
    print(text_read == t)

True
True
True
True


Watch out for this character as it is very awkward;

In [ ]:
print("first print")
print(ord("\x8e"))
print("second print")
print(chr(142))

Identify invalid characters using string:  Don't use this one as the later one is better.

In [ ]:

# Define a string containing only valid characters (alphanumeric + punctuation)
valid_chars = string.ascii_letters + string.digits + string.punctuation + " " = "\n"

# Example text with some invalid characters
#test_text = "This is an example text with invalid characters like Ò, ë, etc."
test_text = text
# Step 1: Identify invalid characters
invalid_chars = set([char for char in test_text if char not in valid_chars])

# Display the invalid characters found
print(f"Invalid characters found: {invalid_chars}")

# Step 2: Create a dictionary or list of tuples for replacement
# For simplicity, let's replace invalid characters with '?'
replacement_dict = {char: '?' for char in invalid_chars}

# If you want to create a list of tuples instead:
replacement_list = [(char, '?') for char in invalid_chars]

# Step 3: Replace invalid characters in the text
corrected_text = test_text
for char, replacement in replacement_dict.items():
    corrected_text = corrected_text.replace(char, replacement)

# Display the corrected text
#print(f"Corrected text: {corrected_text}")

# Output
print("Replacement dictionary:", replacement_dict)
print("Replacement list of tuples:", replacement_list)


Identify invalid characters using regular expressions (re):


In [10]:
# Define the pattern for valid characters (alphanumeric, spaces, and basic punctuation)
valid_char_pattern = re.compile(r'[a-zA-Z0-9\s.:;,?!\'"()£\-—…&]')

re_chars = len(valid_encodings) * [0]

for enc, encoding_index in enumerate(valid_encodings):
    test_text = texts[enc]

# Find invalid characters
    invalid_chars = set()
    for i, char in enumerate(test_text):
        if not valid_char_pattern.match(char):
            invalid_chars.add(char)

# Output the list of invalid characters and their positions
    invalid_char_positions = [(char, [i for i, c in enumerate(test_text) if c == char]) for char in invalid_chars]
    print(f"invalid characters using re for {encodings[encoding_index]}: {invalid_chars}")
#print(invalid_char_positions)
    re_chars[enc] = invalid_chars

invalid characters using re for utf-8: {'‘', '“', '”', 'é', '’'}
invalid characters using re for latin-1: {'\x9c', '\x94', '\x99', '\x98', '©', '\x80', 'Ã', 'â', '\x9d', '¦'}
invalid characters using re for iso-8859-1: {'\x9c', '\x94', '\x99', '\x98', '©', '\x80', 'Ã', 'â', '\x9d', '¦'}
invalid characters using re for iso-8859-15: {'\x9c', '\x94', '\x99', '\x98', '©', '\x80', 'Ã', 'Š', 'â', '\x9d'}


In [11]:
# Define valid characters using the string module
valid_chars = string.ascii_letters + string.digits + string.punctuation + "…—àêé \n\t"
valid_chars = ''.join([char for char in valid_chars if char not in "{}~$%"])
print (f"valid characters used by string: {string.ascii_letters + string.digits + ' ' + string.punctuation}")

string_chars = len(valid_encodings) * [0]

for enc, encoding_index in enumerate(valid_encodings):
    test_text = texts[enc]

    invalid_chars = set()
    for i, char in enumerate(test_text):
        if char not in valid_chars:
            invalid_chars.add(char)
    #invalid_chars.difference_update({"\n","\t"})

    invalid_char_positions = [(char, [i for i, c in enumerate(test_text) if c == char]) for char in invalid_chars]
    print(f"invalid characters found using string for {encodings[encoding_index]}: {invalid_chars}")
#print(invalid_char_positions)
    string_chars[enc] = invalid_chars



valid characters used by string: abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
invalid characters found using string for utf-8: {'\x0c', '‘', '“', '”', '’'}
invalid characters found using string for latin-1: {'\x9c', '\x94', '\x99', '\x98', '©', '\x80', '\x0c', 'Ã', 'â', '\x9d', '¦'}
invalid characters found using string for iso-8859-1: {'\x9c', '\x94', '\x99', '\x98', '©', '\x80', '\x0c', 'Ã', 'â', '\x9d', '¦'}
invalid characters found using string for iso-8859-15: {'\x9c', '\x94', '\x99', '\x98', '©', '\x80', '\x0c', 'Ã', 'Š', 'â', '\x9d'}


In [12]:
for char in invalid_chars:
    print(ord(char)," ", char)


156   
148   
153   
152   
169   ©
128   
12   
195   Ã
352   Š
226   â
157   


In [ ]:
#just a little test:
''.join([char for char in "abcdefgh" if char not in "cd"])

In [ ]:
print(valid_chars)
print(ord(valid_chars[68]))  #this is the ' after the & in the list
print(ord(valid_chars[-13])) #this is the ` before the { in the list
print(ord("'"))
print(ord('"'))
print(ord("`"))

In [13]:
def display_chars(chars, method, unprintable = 142):
    number = len(chars)
    invalid_chars = '  '.join(char for char in chars)
    unprintable_message = " + unprintable character" if chr(unprintable) in invalid_chars else ""
    print(f"{number} invalid chars found using {method}:  {invalid_chars}{unprintable_message}")

In [14]:
for enc, encoding_index in enumerate(valid_encodings):
    print(f"with encoding {encodings[encoding_index]}")
    
    display_chars(re_chars[enc], "re   ")
    display_chars(string_chars[enc], "string")
        
    re_only = re_chars[enc] - string_chars[enc]
    string_only = string_chars[enc] - re_chars[enc]
    
    print(f"\n{len(re_only)} chars in re but not in string:    {'  '.join(char for char in re_only)}")
    print(f"{len(string_only)} chars in string but not in re:    {'  '.join(char for char in string_only)}\n")

with encoding utf-8
5 invalid chars found using re   :  ‘  “  ”  é  ’
5 invalid chars found using string:    ‘  “  ”  ’

1 chars in re but not in string:    é
1 chars in string but not in re:    

with encoding latin-1
10 invalid chars found using re   :          ©    Ã  â    ¦
11 invalid chars found using string:          ©      Ã  â    ¦

0 chars in re but not in string:    
1 chars in string but not in re:    

with encoding iso-8859-1
10 invalid chars found using re   :          ©    Ã  â    ¦
11 invalid chars found using string:          ©      Ã  â    ¦

0 chars in re but not in string:    
1 chars in string but not in re:    

with encoding iso-8859-15
10 invalid chars found using re   :          ©    Ã  Š  â  
11 invalid chars found using string:          ©      Ã  Š  â  

0 chars in re but not in string:    
1 chars in string but not in re:    



In [ ]:
print(ord("é"))
a = "eé".encode('utf-8')
print(len(a))

In [16]:
def compare_texts(text1, text2):
    if len(text1) != len(text2):
        #raise ValueError("Strings must be of the same length!")
        
# Create a list of tuples with differing characters
        differences = [(char1, char2) for char1, char2 in zip(text1, text2) if char1 != char2]

# Print the list of tuples
        print(differences[:100])



In [17]:
def find_first_difference(text_1, text_2):
    i = 0
    max_length = min(len(text_1),len(text_2))
    while ((text_1[:i] == text_2[:i]) and (i < max_length)):
        i = i + 1
    if i < max_length:
        print (f"first text, position {i} is {text_1[i]}")
        print(text_1[i:i+20])
        print (f"second text, position {i} is {text_2[i]}")
        print(text_2[i:i+20])
    else:
        print("no differences up to end of the smaller text")

In [18]:
# compare_texts(texts[0], texts[1])
# find_first_difference(texts[0],texts[1])

Now we are going to display all the invalid characters in their context to decide what to do about them. We will just do it for one of the versions as they are all the same.

In [19]:
def display_invalid_chars(test_text, window_size = 200, max_to_display = 8):
# Find all invalid characters and their positions
    #valid_chars = string.ascii_letters + string.digits + string.punctuation + " —…()£-&\n\t"
    #print(valid_chars)
    invalid_char_positions = [(i, char) for i, char in enumerate(test_text) if char not in valid_chars]
                          
    icp_list_sorted = sorted(invalid_char_positions,key=lambda x: x[1])
    
    for pos, char in icp_list_sorted:
        start = max(pos - window_size // 2, 0)
        end = min(pos + window_size // 2, len(test_text))
        window = test_text[start:end]
    
    # Highlight the invalid character in the window
        highlighted_window = window[:pos-start] + f"[{char}]" + window[pos-start+1:]
        highlighted_window = highlighted_window.replace("\n", " ")
        highlighted_window = highlighted_window.replace("\t", " ")
        print(f"Invalid character '{char}' ({ord(char)}) at position {pos}:\n{highlighted_window}\n")




In [20]:
def display_text_in_context(source_text,target_text,window = 50, repeat = False):
    ind = source_text.find(target_text)
    if not repeat:
        if ind != -1:
            window = source_text[max(0, ind - window): ind + window + len(target_text)]
            window = window.replace("\n", " ")
            window = window.replace("\t", " ")
            print(window)
        print("not found")
    else:
        if ind == -1:
            print("not found")
        while True:
            ind = source_text.find(target_text, ind)
            if ind == -1:
                break
            start_window = max(0, ind - window)
            end_window = min(len(source_text), ind + len(target_text) + window)
            window_text = source_text[start_window:end_window]
            print(ind, window_text)
            ind += 1  # Move to the next position
        
        
        
        
#        while ind != -1:
#            display_window = source_text[max(0, ind - window): ind + window + len(target_text)]
#            display_window = display_window.replace("\n", " ")
#            display_window = display_window.replace("\t", " ")
#            print(display_window)
#            source_text = source_text[ind:]


In [28]:
display_invalid_chars(texts[0][:2000])

Invalid character '' (12) at position 7:
Part I []                            Chapter One   You would have searched a long time for the sort of windi

Invalid character '' (12) at position 1882:
for ease, this is how we will refer to them. I would say this couple lived an isolated life, but in []those days few were “isolated” in any sense we would understand. For warmth and protection, the vil

Invalid character '“' (8220) at position 1903:
 we will refer to them. I would say this couple lived an isolated life, but in those days few were [“]isolated” in any sense we would understand. For warmth and protection, the villagers lived in sh

Invalid character '”' (8221) at position 1912:
refer to them. I would say this couple lived an isolated life, but in those days few were “isolated[”] in any sense we would understand. For warmth and protection, the villagers lived in sh



In [24]:
# display_invalid_chars(texts[1][0:5000])

In [32]:
display_invalid_chars(texts[2])

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [33]:
display_invalid_chars(texts[3])

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
print("€" in texts[3])
print(texts[3][246245])

In [29]:
# Function to find and replace specified strings in a text
def replace_strings_with_context(text, replacements, window=10, silent = False):
    """
    Replaces specified strings within a text and prints the context around each replacement.
    
    Parameters:
    - text: The original string where replacements are done.
    - replacements: A dictionary where keys are the substrings to find, and values are their replacements.
    - window: Number of characters to show before and after the replaced string for context.
    
    Returns:
    - A modified string with the replacements applied.
    """
    counter = 0
    for old, new in replacements.items():
        index = text.find(old)
        while index != -1:
            # Extract context around the found string
            start = max(index - window, 0)
            end = min(index + len(old) + window, len(text))
            before = text[start:index]
            after = text[index + len(old):end]
            before = before.replace("\n", " ")
            before = before.replace("\t", " ")
            after = after.replace("\n", " ")
            after = after.replace("\t", " ")

            # Print before, replaced, and after strings
            if not silent:
                print(f"Before: {before}[{old}]{after}")
                print(f"After:  {before}[{new}]{after}\n")

            # Replace the string in the text
            text = text[:index] + new + text[index + len(old):]
            counter += 1
            
            # Find the next occurrence of the old string
            index = text.find(old, index + len(new))

    print(f"made {counter} changes")
    return text




In [ ]:
print(ord("b"))
print(chr(98))
print(ord("Ž"))

In [30]:
replacements = dict()
# replacements["t}:le"] = "the"
# replacements["up o~ the"] = "up on the"
# replacements["the ~ort"] = "the sort"
# replacements["than ~ might"] = "than might"
# replacements["the house~"] = "the house."
# replacements["while ~earlier"] = "while earlier"
# replacements["meetings ~once"] = "meetings once"
# replacements["Ž"] = "é"
# replacements["mean- it"] = "mean it"
# replacements["¤"] = "s"
# replacements["Ð"] = "-"
# replacements["É"] = '-'
# replacements["Ó"] = '"'
# replacements["Ò"] = '"'
# replacements["Ô"] = '"'
# replacements["Rover.Ó I thanked"] = "Rover  I thanked"
# replacements['Rover." I thanked'] = "Rover  I thanked"
# replacements["Õ"] = "'"
# replacements[chr(136)] = "à"
# replacements[chr(142)] = "é"
# replacements[chr(144)] = "ê"
# replacements["Ñ"] = "-"
replacements[chr(12)] = ""
# replacements["%egan"] = "began"
# replacements["t4arijço"]="Sachiko"
replacements[chr(8217)] = "'"
replacements[chr(8216)] = '"'
replacements[chr(8220)] = '"'
replacements[chr(8221)] = '"'
# replacements["*"] = '"'
# replacements["AH the same"] = "Ah the same"
# replacements["•"] = ""

for key, value in replacements.items():
        print(f"{key}: {value}")

: 
’: '
‘: "
“: "
”: "


In [31]:
text_edited = replace_strings_with_context(texts[0],replacements, silent = True)

made 7483 changes


Now see if there are any problems left:

In [32]:
display_invalid_chars(text_edited)


In [33]:
display_text_in_context(text_edited, "Chapter", window = 40, repeat = True)
display_text_in_context(text_edited, "PART", window = 80, repeat = True)

35 Part I
                            Chapter One


You would have searched a long ti
44865 ing today."
                            Chapter Two


There were, however, plenty of th
82085 ese worries."
                          Chapter Three


The Saxon village, viewed from 
150826 ay or two."
                            Chapter Four


Never in his life had he seen hi
169687 uld tremble.
                           Chapter Five


After a punishing climb for much
227857 ly."
Part II
                           Chapter Six


For all his tiredness, Axl was fi
287030 ace
radiant.
                           Chapter Seven


A hand had been shaking him, bu
328651 t keeps us."
                           Chapter Eight


The young monk was a thin, sick
386538 awaits us.
                             Chapter Ten


He had never intended to deceive 
400611 ft a sword?'
                           Chapter Eleven


He longed for a patch of sun t
421916 my shoulders."
                         Chapter Twelve

In [104]:
print(ord("\n"))

10


In [37]:
display_text_in_context(text_edited, "Part III", window = 200)

rt, honestly and frankly, it was possible Wistan would understand and give him another chance.    A shoe in each hand, Edwin sprang lightly towards the next rock, faintly cheered by this possibility. Part III                      Gawain's First Reverie   Those dark widows. For what purpose did God place them on this mountain path before me? Does he wish to test my humility? Is it not enough he watches me 
not found


In [43]:
display_text_in_context(text_edited, "Gawain's First Reverie", window = 200)


as possible Wistan would understand and give him another chance.    A shoe in each hand, Edwin sprang lightly towards the next rock, faintly cheered by this possibility. Part III                      Gawain's First Reverie   Those dark widows. For what purpose did God place them on this mountain path before me? Does he wish to test my humility? Is it not enough he watches me save that gentle couple, the wounded boy als
not found


In [44]:
titles = list()
# titles.append(("PART ONE", "Chapter One"))
# titles.append(("PART TWO", "Chapter Four"))
# titles.append(("PART THREE", "Chapter Ten"))
# titles.append(("PART FOUR", "Chapter Twelve"))
# titles.append(("PART FIVE", "Chapter Fourteen"))
# titles.append(("PART SIX", "Chapter Sixteen"))
# titles.append(("PART SEVEN", "Chapter Twenty-three"))
titles.append(("Part I", "Chapter One"))
titles.append(("Part II", "Chapter Six"))
titles.append(("Part III", "Gawain's First Reverie"))                            
titles.append(("Part IV", "Chapter Fifteen"))



for edit_start, edit_end in titles:
    print(edit_start, edit_end)


Part I Chapter One
Part II Chapter Six
Part III Gawain's First Reverie
Part IV Chapter Fifteen


In [45]:
def remove_chunks(text, chunks, remove_all = False, buffer = 100):
    # removes chunks defines by a list of tuples, start of chunk to end of chunk, removes the second part only if remove_all is set to True
    print("text length:", len(text))
    for edit_start, edit_end in chunks: 
        ind1 = text.find(edit_start)
        ind2 = text.find(edit_end)
        if remove_all:
            ind2 += len(edit_end)
        if ind2 > ind1 and ind2 < ind1 + 100 and ind1 != -1 and ind2 != -1:
            print (f"removing {edit_start} to {edit_end}")
            text = text[:ind1] + text[ind2:]
        else:
            print(f"{edit_start} to {edit_end} not removed")
    print("text length reduced to: ", len(text))
    return text


In [90]:
#TEST
tt = "The quick brown fox jumps over the lazy dog"
tt1 = [("quick", "the ")]
tt2 = remove_chunks(tt, tt1, remove_all = True)
print(tt)
print(tt2)

removing quick to the 
The quick brown fox jumps over the lazy dog
The lazy dog


In [46]:
text_edited_1 = remove_chunks(text_edited, titles, remove_all = True)

text length: 569032
removing Part I to Chapter One
removing Part II to Chapter Six
removing Part III to Gawain's First Reverie
removing Part IV to Chapter Fifteen
text length reduced to:  568840


In [47]:
tidy_up = dict()
tidy_up["  "] = " " 
tidy_up["Chapter Sixteen"] = ""
tidy_up["Chapter Seventeen"] = ""
tidy_up["Chapter Eighteen"] = ""
tidy_up["Chapter Nineteen"] = ""
tidy_up["Chapter Two"] = ""
tidy_up["Chapter Three"] = ""
tidy_up["Chapter Four"] = ""
tidy_up["Chapter Five"] = ""
tidy_up["Chapter Six"] = ""
tidy_up["Chapter Seven"] = ""
tidy_up["Chapter Eight"] = ""
tidy_up["Chapter Nine"] = ""
tidy_up["Chapter Ten"] = ""
tidy_up["Chapter Eleven"] = ""
tidy_up["Chapter Twelve"] = ""
tidy_up["Chapter Thirteen"] = ""
tidy_up["Chapter Fourteen"] = ""
tidy_up["Chapter Fifteen"] = ""
tidy_up["Chapter Twenty-one"] = ""
tidy_up["Chapter Twenty-two"] = ""
tidy_up["Chapter Twenty"] = ""


In [49]:
# text_edited_2 = replace_strings_with_context(text_edited_1,tidy_up, silent = False)

In [55]:
display_text_in_context(text_edited_2, "14")

not found


In [56]:
def find_capitalized_words(text):
    """
    Finds all words in the string that are in capitals, excluding words of one character.
    
    Parameters:
    - text: The input string.
    
    Returns:
    - A list of fully capitalized words with more than one character.
    """
    # Use regex to match words with all capital letters and exclude single-character words
    capitalized_words = re.findall(r'\b[A-Z]{2,}\b', text)
    return capitalized_words


In [57]:
capitalized_words = find_capitalized_words(text_edited_2)
for word in capitalized_words:
    if word != "OK":
        ind = text_edited.find(word)
        window = text_edited[ind-75:ind+40]
        window = window.replace("\n", " ")
        window = window.replace("\t", " ")
        print(window)


In [58]:
with open(file_path_edited_2, 'w', encoding='utf-8') as f:
    f.write(text_edited_2)

Check that saving and loading the corrected file doesn't change it.  There will be a change when it encounters é because it takes 2 bytes.  Up to that point, there should be no differences.

In [59]:
with open(file_path_edited_2, 'r', encoding='utf-8') as f:
    text = f.read()

In [60]:
text == text_edited_2

True

In [114]:
text_edited.find("é")

-1

In [113]:
print(len(text), len(text_edited_2))

578625 578369


In [115]:
text_edited_2.find(chr(8216))

-1

In [120]:
text_edited_1.find(chr(8230))

19068

In [122]:
print(text_edited_1[200:230])
print(text_edited_1[19000:19120])
      

nese name, and I— perhaps out 
f the
city.'
"Oh really? I didn't realize that. You lived there with…with friends?"
Sachiko stopped pouring and looked a


In [61]:

# Sample text with hyphenated words across lines
t = texts[0]
# Use regex to find words that are hyphenated across lines
# Match sequences where a hyphen is at the end of a line, followed by a newline, and then continued with a word
hyphenated_words = re.findall(r"(\w+)-\n(\w+)", t)

# Print the hyphenated words
for first_part, second_part in hyphenated_words:
    print(f"Found broken word: {first_part}-{second_part}")
print(len(hyphenated_words))
print(len(t))


Found broken word: foot-path
Found broken word: mist-layered
Found broken word: half-remembered
Found broken word: half-wit
Found broken word: slow-witted
Found broken word: near-certainty
Found broken word: trap-door
Found broken word: foul-smelling
Found broken word: she-dragon
Found broken word: well-guarded
Found broken word: near-darkness
Found broken word: long-handled
Found broken word: she-dragon
Found broken word: she-dragon
Found broken word: she-dragon
Found broken word: near-silhouette
Found broken word: she-dragon
Found broken word: she-dragon
Found broken word: she-dragon
19
569311
